In [1]:
pip install pandas pyodbc

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import pyodbc
import csv

# Function to create tables and load data from CSV files
def setup_database():
    # Connect to SQL Server database
    conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=MSI-GF63\\SQLEXPRESS;DATABASE=bookstore;Trusted_Connection=yes;')
    cursor = conn.cursor()
    
    # Define schema creation queries
    schema_queries = [
        """
        IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = N'bookstore')
        BEGIN
            CREATE DATABASE bookstore;
        END
        """,
        """
        USE bookstore;
        """,
        """
        IF OBJECT_ID('customers', 'U') IS NOT NULL DROP TABLE customers;
        IF OBJECT_ID('invoices', 'U') IS NOT NULL DROP TABLE invoices;
        IF OBJECT_ID('invoice_lines', 'U') IS NOT NULL DROP TABLE invoice_lines;
        """,
        """
        CREATE TABLE customers (
            id INT PRIMARY KEY,
            name VARCHAR(255),
            email VARCHAR(255),
            tel VARCHAR(20),
            created_at DATETIME,
            updated_at DATETIME
        );
        """,
        """
        CREATE TABLE invoices (
            id INT PRIMARY KEY,
            number VARCHAR(20),
            sub_total DECIMAL(10, 2),
            tax_total DECIMAL(10, 2),
            total DECIMAL(10, 2),
            customer_id INT,
            created_at DATETIME,
            updated_at DATETIME,
            FOREIGN KEY (customer_id) REFERENCES customers(id)
        );
        """,
        """
        CREATE TABLE invoice_lines (
            id INT PRIMARY KEY,
            description VARCHAR(255),
            unit_price DECIMAL(10, 2),
            quantity INT,
            sub_total DECIMAL(10, 2),
            tax_total DECIMAL(10, 2),
            total DECIMAL(10, 2),
            tax_id INT,
            sku_id INT,
            invoice_id INT,
            FOREIGN KEY (invoice_id) REFERENCES invoices(id)
        );
        """
    ]

    # Execute schema creation queries
    for query in schema_queries:
        cursor.execute(query)

    # Commit changes and close connection
    conn.commit()

    with open('customers.csv','r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        cursor.executemany("INSERT INTO customers VALUES (?, ?, ?, ?, ?, ?)", reader)

    with open('invoices.csv', 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        cursor.executemany("INSERT INTO invoices VALUES (?, ?, ?, ?, ?, ?, ?, ?)", reader)

    with open('invoice_lines.csv', 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        cursor.executemany("INSERT INTO invoice_lines VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", reader)

    # Commit data changes and close connection
    conn.commit()
    conn.close()
